### Proximal policy optimization algorithm from scratch

In [71]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import gymnasium as gym
import torch.distributions as dist
from enum import Enum

class PolicyNetwork(nn.Module):
    def __init__(self, state_space_dim, action_space_dim, is_continuous):
        super().__init__()
        self.is_continuous = is_continuous
        self.state_dim = state_space_dim
        self.action_space_dim = action_space_dim
        self.fc1 = nn.Linear(state_space_dim, 64)
        self.fc2 = nn.Linear(64, 64)
        # For continuous action dim, return mean and only diagonal entries on covariance matrix since action states are probably independent
        self.fc3 = nn.Linear(64, 2 * action_space_dim if is_continuous else action_space_dim)

    def forward(self, state):
        x = F.relu(self.fc1(state))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        if self.is_continuous:
            mean = x[..., :self.action_space_dim]
            covar = torch.exp(x[..., self.action_space_dim:])
            return mean, torch.diag(covar)
        else:
            return torch.softmax(x, dim=-1)


class BaselineVNetwork(nn.Module):
    def __init__(self, state_space_dim, max_timesteps):
        super().__init__()
        self.max_timesteps = max_timesteps
        self.fc1 = nn.Linear(state_space_dim + 1, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 1)

    def forward(self, state, timestep):
        timestep = timestep.float() / self.max_timesteps
        x = F.relu(self.fc1(torch.cat([state, timestep.unsqueeze(-1)])))
        x = F.relu(self.fc2(x))
        return self.fc3(x)


class AdvantageEstimation(Enum):
    VPG = 0
    # Use k=5 step lookahead
    A2C = 1


# Implement vpg alg
iterations = 100
max_timesteps = 500
t_per_iter = 50
env = gym.make('CartPole-v1', render_mode='human', max_episode_steps=max_timesteps)
state_space_dim = 4
action_space_dim = 2
policy = PolicyNetwork(state_space_dim, action_space_dim, False)
policy_opt = torch.optim.AdamW(policy.parameters(), lr=5e-4)
policy_opt_steps = 5
baseline = BaselineVNetwork(state_space_dim, max_timesteps)
baseline_opt = torch.optim.AdamW(baseline.parameters(), lr=1e-3)
baseline_opt_steps = 10
discount = 0.99
adv_est_scheme = AdvantageEstimation.A2C
eps = 0.2

In [73]:
for iter in range(100):
    # Collect a set of trajectories by executing the current policy
    trajectories = []
    with torch.no_grad():
        for t_ind in range(t_per_iter):
            current_state, _ = env.reset()
            current_state = torch.tensor(current_state, dtype=torch.float32)
            traj = []
            for _ in range(max_timesteps):
                actions_dist = policy(current_state)
                if policy.is_continuous:
                    mean, covar = actions_dist
                    mgd = dist.MultivariateNormal(mean, covar)
                    action = mgd.sample()
                else:
                    d = dist.Categorical(actions_dist)
                    action = d.sample()

                next_state, reward, terminated, truncated, _ = env.step(action.item())
                new_step = (current_state, action, reward)
                traj.append(new_step)
                current_state = torch.tensor(next_state, dtype=torch.float32)

                if terminated or truncated:
                    break

            trajectories.append(traj)


    # At each timestep in each trajectory, compute the return and advantage estimate
    def calc_returns():
        if adv_est_scheme == AdvantageEstimation.VPG:
            returns = [[sum(discount**(t_prime - time) * traj[t_prime][2] for t_prime in range(time, len(traj))) for time in range(len(traj))] for traj in trajectories]
        else:
            returns = []
            for traj in trajectories:
                traj_returns = []
                for time in range(len(traj)):
                    t_return = 0
                    is_long_en = len(traj) > time + 5
                    for t_prime in range(time, min(len(traj), time + 5)):
                        t_return += discount**(t_prime - time) * traj[t_prime][2]

                    if is_long_en:
                        t_return += discount**5 * baseline(traj[time + 5][0], torch.tensor(time + 5, dtype=torch.float32))

                    traj_returns.append(t_return)

                returns.append(traj_returns)

        return returns

    def calc_baselines():
       return [[baseline(traj[time][0], torch.tensor(time, dtype=torch.float32)) for time in range(len(traj))] for traj in trajectories]

    def calc_adv_ests(returns, log=False):
        baselines = calc_baselines()
        return [[(returns[traj_ind][step_ind] - baselines[traj_ind][step_ind]) for step_ind in range(len(trajectories[traj_ind]))] for traj_ind in range(len(trajectories))]

    def normalize_adv_ests(adv_ests):
        flattened = torch.stack([a for t in adv_ests for a in t])
        return [[(ae - flattened.mean()) / (flattened.std() + 1e-8) for ae in t_ae] for t_ae in adv_ests]

    def calc_baseline_loss():
        returns = calc_returns()
        adv_ests = calc_adv_ests(returns)
        adv_ests = [a for traj_a in adv_ests for a in traj_a]
        return (torch.stack(adv_ests) ** 2).sum() / len(trajectories)

    def collect_state_act_prob():
        vals = []
        for traj in trajectories:
            for state, action, _ in traj:
                if policy.is_continuous:
                    mean, covar = policy(state)
                    d = dist.MultivariateNormal(mean, covar)
                else:
                    d = dist.Categorical(policy(state))
                # Add log pdf val for now for numerical stability and consistency
                vals.append(d.log_prob(action))
        return vals

     # Update policy by maximizing PPO objective
    with torch.no_grad():
        returns = calc_returns()
        adv_ests = calc_adv_ests(returns, log=True)
        flattened_adv_ests = torch.stack([a for adv_est in adv_ests for a in adv_est])

    with torch.no_grad():
        old_policy_dist_vals = torch.stack(collect_state_act_prob())

    policy_losses = []
    for i in range(5):
        policy_dist_vals = torch.stack(collect_state_act_prob())
        ratios = torch.exp(policy_dist_vals - old_policy_dist_vals)
        clipped_ratios = torch.clamp(ratios, 1 - eps, 1 + eps)

        temp = []
        for m in range(len(ratios)):
            temp.append(torch.min(ratios[m] * flattened_adv_ests[m], clipped_ratios[m] * flattened_adv_ests[m]))

        temp = torch.stack(temp)
        policy_loss = -temp.mean()
        policy_losses.append(policy_loss)
        policy_opt.zero_grad()
        policy_loss.backward()
        policy_opt.step()


    # Re-fit the baseline
    last_baseline_loss = 0
    for i in range(baseline_opt_steps):
        baseline_loss = calc_baseline_loss()
        last_baseline_loss = baseline_loss
        baseline_opt.zero_grad()
        baseline_loss.backward()
        baseline_opt.step()


    print("--------------------")
    print("Iteration:", iter)
    print("Avg policy loss:", torch.stack(policy_losses).mean())
    print("Last baseline loss:", last_baseline_loss)
    print("Avg return:", sum(r[0] for r in returns) / len(returns))
    print("Avg steps:", sum(len(t) for t in trajectories) / len(trajectories))
    print("--------------------\n\n")


--------------------
Iteration: 0
Avg policy loss: tensor(-4.3689, grad_fn=<MeanBackward0>)
Last baseline loss: tensor(430.2012, grad_fn=<DivBackward0>)
Avg return: tensor([5.1840])
Avg steps: 21.6
--------------------


--------------------
Iteration: 1
Avg policy loss: tensor(-4.3169, grad_fn=<MeanBackward0>)
Last baseline loss: tensor(410.7805, grad_fn=<DivBackward0>)
Avg return: tensor([5.3130])
Avg steps: 21.26
--------------------


--------------------
Iteration: 2
Avg policy loss: tensor(-4.2887, grad_fn=<MeanBackward0>)
Last baseline loss: tensor(421.9746, grad_fn=<DivBackward0>)
Avg return: tensor([5.4716])
Avg steps: 22.32
--------------------


--------------------
Iteration: 3
Avg policy loss: tensor(-4.2046, grad_fn=<MeanBackward0>)
Last baseline loss: tensor(403.2169, grad_fn=<DivBackward0>)
Avg return: tensor([5.7169])
Avg steps: 22.22
--------------------


--------------------
Iteration: 4
Avg policy loss: tensor(-4.0333, grad_fn=<MeanBackward0>)
Last baseline loss: t

KeyboardInterrupt: 

### Reflections

For the most part, since I had a working vpg + a2c algorithm, implementing ppo didn't take too long. One thing that shocked me though was how fast and large the improvements would be! By 40 iterations, ppo had already solved Cartpole (averaging 300 steps). Also at times, the steps would increase by +80 between iterations. For contrast, with vpg the learning was very chaotic and it took ~200 steps for it to reach 300 avg steps.